In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Prediktivní modelování - metody a implementace

Tento notebook pokrývá tři hlavní přístupy k prediktivnímu modelování časových řad:
1. **ARIMA modely** - statistický přístup
2. **Exponenciální vyhlazování** (Holt-Winters) - klasická metoda
3. **Machine Learning** - Random Forest s feature engineering

In [5]:
# Import základních knihoven pro práci s časovými řadami
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Nastavení pro české lokalizace a lepší zobrazení grafů
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

In [ ]:
# Načtení datasetu energetické spotřeby
df_energy = pd.read_csv('/content/drive/MyDrive/service-brain-digital-horizon/WORKSPACE/2025-07-25/energy_consumption_timeseries.tsv',
                        sep='\t', on_bad_lines='skip')

# Vytvoření datetime indexu z datumu a hodiny
df_energy['datetime'] = pd.to_datetime(df_energy['date'] + ' ' + df_energy['hour'].astype(str) + ':00:00')
df_energy = df_energy.set_index('datetime').sort_index()

# Základní informace o datasetu
print(f"Dataset obsahuje {len(df_energy)} záznamů")
print(f"Časové období: {df_energy.index.min()} až {df_energy.index.max()}")
print("\nPrvních 5 řádků:")
df_energy.head()

## 1. ARIMA modelování

ARIMA (AutoRegressive Integrated Moving Average) je statistická metoda pro predikci časových řad.

In [ ]:
# Funkce pro test stacionarity (klíčový předpoklad ARIMA)
def test_stacionarity(timeseries, nazev="časová řada"):
    """Augmented Dickey-Fuller test pro testování stacionarity"""
    result = adfuller(timeseries)
    print(f'Test stacionarity pro {nazev}:')
    print(f'ADF Statistika: {result[0]:.4f}')
    print(f'p-hodnota: {result[1]:.4f}')

    if result[1] <= 0.05:
        print("✓ Řada JE stacionární (p-hodnota ≤ 0.05)")
        return True
    else:
        print("✗ Řada NENÍ stacionární (p-hodnota > 0.05)")
        return False

# Test původních dat
ts_data = df_energy['energy_consumption_mwh'].dropna()
is_stationary = test_stacionarity(ts_data, "spotřeba energie")

In [8]:
# Pokud data nejsou stacionární, aplikujeme diferencování
if not is_stationary:
    ts_diff = ts_data.diff().dropna()
    print("\nPo diferencování:")
    test_stacionarity(ts_diff, "diferencovaná spotřeba")

    # Vizualizace původních a diferencovaných dat
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

    ax1.plot(ts_data.index, ts_data)
    ax1.set_title('Původní data - spotřeba energie')
    ax1.set_ylabel('MWh')

    ax2.plot(ts_diff.index, ts_diff)
    ax2.set_title('Diferencovaná data')
    ax2.set_ylabel('Změna MWh')

    plt.tight_layout()
    plt.show()

In [ ]:
# Trénování ARIMA modelu
def trenuj_arima_model(data, order=(2,1,2)):
    """Natrénuje ARIMA model s danými parametry"""
    print(f"Trénování ARIMA{order} modelu...")

    model = ARIMA(data, order=order)
    fitted_model = model.fit()

    # Základní statistiky modelu
    print(f"AIC: {fitted_model.aic:.2f}")
    print(f"BIC: {fitted_model.bic:.2f}")

    return fitted_model

# Natrénujeme ARIMA model
arima_model = trenuj_arima_model(ts_data, order=(2,1,2))

# Prognóza na 24 hodin dopředu
arima_forecast = arima_model.forecast(steps=24)
print(f"\nPrognóza na příštích 24 hodin:")
print(f"Průměrná předpovídaná spotřeba: {arima_forecast.mean():.1f} MWh")

## 2. Exponenciální vyhlazování (Holt-Winters)

Holt-Winters metoda dokáže zachytit trend i sezónnost v datech.

In [ ]:
# Holt-Winters exponenciální vyhlazování
print("Trénování Holt-Winters modelu...")

# Model s additívní sezónností (24 hodin = denní cyklus)
hw_model = ExponentialSmoothing(
    ts_data,
    trend='add',        # Additivní trend
    seasonal='add',     # Additivní sezónnost
    seasonal_periods=24 # 24-hodinový cyklus
).fit()

# Prognóza na 24 hodin
hw_forecast = hw_model.forecast(24)

print(f"Holt-Winters prognóza - průměr: {hw_forecast.mean():.1f} MWh")
print(f"Rozsah prognózy: {hw_forecast.min():.1f} - {hw_forecast.max():.1f} MWh")

## 3. Machine Learning přístup - Random Forest

Přeformulujeme časovou řadu na supervised learning problém pomocí feature engineering.

In [ ]:
# Feature engineering - vytvoření příznaků pro ML model
def vytvor_ml_priznaky(df):
    """Vytvoří lag a rolling features pro ML model"""
    df_ml = df.copy()

    # Lag features (hodnoty z předchozích časových okamžiků)
    for lag in [1, 2, 3, 6, 12, 24]:
        df_ml[f'lag_{lag}h'] = df_ml['energy_consumption_mwh'].shift(lag)

    # Rolling statistics (klouzavé průměry a směrodatné odchylky)
    for window in [3, 6, 12, 24]:
        df_ml[f'rolling_mean_{window}h'] = df_ml['energy_consumption_mwh'].rolling(window).mean()
        df_ml[f'rolling_std_{window}h'] = df_ml['energy_consumption_mwh'].rolling(window).std()

    # Cyklické features pro čas (zachycení denního cyklu)
    df_ml['hour_sin'] = np.sin(2 * np.pi * df_ml.index.hour / 24)
    df_ml['hour_cos'] = np.cos(2 * np.pi * df_ml.index.hour / 24)
    df_ml['day_sin'] = np.sin(2 * np.pi * df_ml.index.dayofweek / 7)
    df_ml['day_cos'] = np.cos(2 * np.pi * df_ml.index.dayofweek / 7)

    return df_ml.dropna()

# Vytvoříme features
ml_data = vytvor_ml_priznaky(df_energy)
print(f"Vytvořeno {len(ml_data.columns)} příznaků pro ML model")
print(f"Dataset po feature engineering: {ml_data.shape}")

In [ ]:
# Příprava dat pro Random Forest
# Vybereme pouze numerické sloupce (vynecháme 'date', 'day_type', 'season')
feature_cols = [col for col in ml_data.columns
                if col not in ['energy_consumption_mwh', 'date', 'day_type', 'season']]

X = ml_data[feature_cols]
y = ml_data['energy_consumption_mwh']

print(f"Počet příznaků: {len(feature_cols)}")
print(f"Příklady příznaků: {feature_cols[:5]}")

# Time Series Cross-Validation (respektuje časové pořadí)
tscv = TimeSeriesSplit(n_splits=3)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

print("\nTrénování Random Forest s křížovou validací:")
mae_scores = []

for i, (train_idx, test_idx) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    rf_model.fit(X_train, y_train)
    predictions = rf_model.predict(X_test)

    mae = mean_absolute_error(y_test, predictions)
    mae_scores.append(mae)
    print(f"Fold {i+1} - MAE: {mae:.2f} MWh")

print(f"\nPrůměrná MAE: {np.mean(mae_scores):.2f} ± {np.std(mae_scores):.2f} MWh")

## 4. Správné porovnání modelů na stejném testovacím setu

**KLÍČOVÉ:** Všechny modely musíme natrénovat na stejných datech a otestovat na stejných datech pro spravedlivé porovnání.

In [ ]:
# KROK 1: Rozdělení dat na train/test
# Posledních 24 hodin použijeme jako test set
test_size = 24
train_data = ts_data[:-test_size]  # Trénovací data (vše kromě posledních 24h)
test_data = ts_data[-test_size:]   # Testovací data (posledních 24h)

print(f"Trénovací data: {len(train_data)} hodin")
print(f"Testovací data: {len(test_data)} hodin")
print(f"Test période: {test_data.index[0]} až {test_data.index[-1]}")

In [ ]:
# KROK 2: Přetrénování všech modelů pouze na trénovacích datech

print("Trénování modelů pouze na trénovacích datech...\n")

# 2A) ARIMA model na train datech
arima_model_final = ARIMA(train_data, order=(2,1,2)).fit()
arima_predictions = arima_model_final.forecast(steps=test_size)
print("✓ ARIMA model natrénován")

# 2B) Holt-Winters na train datech
# Kontrola dostupnosti dat pro sezónní model
seasonal_periods = 24  # 24-hodinový cyklus
min_required_data = 2 * seasonal_periods  # Minimálně 2 kompletní cykly

if len(train_data) >= min_required_data:
    hw_model_final = ExponentialSmoothing(
        train_data,
        trend='add',
        seasonal='add',
        seasonal_periods=seasonal_periods
    ).fit()
    print("✓ Holt-Winters model natrénován (s plnou sezónností)")
else:
    # Fallback: použijeme kratší sezónní období nebo žádné
    print(f"⚠️ Nedostatek dat pro 24h sezónnost ({len(train_data)} < {min_required_data})")
    if len(train_data) >= 24:  # Aspoň 1 den
        hw_model_final = ExponentialSmoothing(
            train_data,
            trend='add',
            seasonal='add',
            seasonal_periods=12  # Zkrácená sezónnost na 12h
        ).fit()
        print("✓ Holt-Winters model natrénován (12h sezónnost)")
    else:
        # Pouze trend, bez sezónnosti
        hw_model_final = ExponentialSmoothing(
            train_data,
            trend='add'
        ).fit()
        print("✓ Holt-Winters model natrénován (pouze trend)")

hw_predictions = hw_model_final.forecast(test_size)

# 2C) Random Forest na train datech
# Musíme vytvořit features i pro train/test split
try:
    ml_data_train = vytvor_ml_priznaky(df_energy[:-test_size])
    ml_data_test = vytvor_ml_priznaky(df_energy)[-test_size:]  # Posledních 24h s features

    X_train_final = ml_data_train[feature_cols]
    y_train_final = ml_data_train['energy_consumption_mwh']
    X_test_final = ml_data_test[feature_cols]

    rf_model_final = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model_final.fit(X_train_final, y_train_final)
    rf_predictions = rf_model_final.predict(X_test_final)
    print("✓ Random Forest model natrénován")

except Exception as e:
    print(f"⚠️ Problém s Random Forest: {e}")
    # Fallback: jednoduchá predikce posledními hodnotami
    rf_predictions = np.full(test_size, train_data.mean())
    print("✓ Random Forest nahrazen průměrnou hodnotou")

In [ ]:
# KROK 3: Výpočet metrik na stejném test setu
print("Porovnání výkonnosti modelů:")
print("=" * 40)

# ARIMA metriky
arima_mae = mean_absolute_error(test_data, arima_predictions)
arima_rmse = np.sqrt(mean_squared_error(test_data, arima_predictions))
print(f"{'ARIMA':15} MAE: {arima_mae:6.2f} MWh | RMSE: {arima_rmse:6.2f} MWh")

# Holt-Winters metriky
hw_mae = mean_absolute_error(test_data, hw_predictions)
hw_rmse = np.sqrt(mean_squared_error(test_data, hw_predictions))
print(f"{'Holt-Winters':15} MAE: {hw_mae:6.2f} MWh | RMSE: {hw_rmse:6.2f} MWh")

# Random Forest metriky
rf_mae = mean_absolute_error(test_data, rf_predictions)
rf_rmse = np.sqrt(mean_squared_error(test_data, rf_predictions))
print(f"{'Random Forest':15} MAE: {rf_mae:6.2f} MWh | RMSE: {rf_rmse:6.2f} MWh")

print("\n" + "=" * 50)
print("VYSVĚTLENÍ VÝSLEDKŮ:")
print("=" * 50)

if rf_mae < arima_mae and rf_mae < hw_mae:
    print("🏆 Random Forest má nejlepší výsledky, protože:")
    print("   • Využívá více příznaků (teplota, lag values, rolling stats)")
    print("   • Dokáže zachytit nelineární vztahy")
    print("   • Feature engineering pomáhá modelu porozumět vzorům")

print(f"\n📊 Rozdíl v přesnosti:")
print(f"   • Random Forest je o {arima_mae-rf_mae:.1f} MWh přesnější než ARIMA")
print(f"   • Random Forest je o {hw_mae-rf_mae:.1f} MWh přesnější než Holt-Winters")

In [ ]:
# KROK 4: Vizualizace porovnání všech modelů
plt.figure(figsize=(15, 10))

# Historická data (posledních 48 hodin pro kontext)
context_data = ts_data[-48:-24]  # 24 hodin před testem
plt.plot(context_data.index, context_data,
         label='Historická data', color='black', linewidth=2)

# Skutečné hodnoty v testovacím období
plt.plot(test_data.index, test_data,
         label='Skutečné hodnoty (test)', color='black',
         linewidth=3, marker='o', markersize=4)

# Predikce jednotlivých modelů
plt.plot(test_data.index, arima_predictions,
         label=f'ARIMA (MAE: {arima_mae:.1f})',
         color='blue', linestyle='--', linewidth=2, marker='s', markersize=3)

plt.plot(test_data.index, hw_predictions,
         label=f'Holt-Winters (MAE: {hw_mae:.1f})',
         color='red', linestyle=':', linewidth=2, marker='^', markersize=3)

plt.plot(test_data.index, rf_predictions,
         label=f'Random Forest (MAE: {rf_mae:.1f})',
         color='green', linestyle='-.', linewidth=2, marker='d', markersize=3)

# Vertikální čára oddělující train/test
plt.axvline(x=train_data.index[-1], color='gray', linestyle='-', alpha=0.7,
            label='Začátek testovacího období')

# Zvýraznění testovací oblasti
plt.axvspan(test_data.index[0], test_data.index[-1],
            alpha=0.1, color='yellow', label='Testovací období (24h)')

plt.title('Porovnání prediktivních modelů na testovacím setu (24 hodin)',
          fontsize=14, fontweight='bold')
plt.xlabel('Čas')
plt.ylabel('Spotřeba energie (MWh)')
plt.legend(loc='upper left', fontsize=10)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n📈 INTERPRETACE GRAFU:")
print("• Černá čára s kolečky = skutečné hodnoty, které chceme předpovědět")
print("• Barevné čáry = predikce jednotlivých modelů")
print("• Čím blíže je barevná čára k černé, tím je model přesnější")
print(f"• Zelená čára (Random Forest) je nejblíže → nejpřesnější model")

In [ ]:
# KROK 5: Bar chart pro vizuální porovnání metrik
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

modely = ['ARIMA', 'Holt-Winters', 'Random Forest']
mae_values = [arima_mae, hw_mae, rf_mae]
rmse_values = [arima_rmse, hw_rmse, rf_rmse]
colors = ['blue', 'red', 'green']

# MAE graf
bars1 = ax1.bar(modely, mae_values, color=colors, alpha=0.7)
ax1.set_title('Mean Absolute Error (MAE)')
ax1.set_ylabel('MAE (MWh)')
ax1.grid(True, alpha=0.3)

# Přidání hodnot na sloupce
for bar, value in zip(bars1, mae_values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 5,
             f'{value:.1f}', ha='center', va='bottom', fontweight='bold')

# RMSE graf
bars2 = ax2.bar(modely, rmse_values, color=colors, alpha=0.7)
ax2.set_title('Root Mean Square Error (RMSE)')
ax2.set_ylabel('RMSE (MWh)')
ax2.grid(True, alpha=0.3)

# Přidání hodnot na sloupce
for bar, value in zip(bars2, rmse_values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 5,
             f'{value:.1f}', ha='center', va='bottom', fontweight='bold')

plt.suptitle('Porovnání výkonnosti modelů na testovacích datech',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 ZÁVĚR:")
print(f"Random Forest dosáhl nejlepší výkonnosti s MAE {rf_mae:.1f} MWh")
print(f"To je o {((arima_mae-rf_mae)/arima_mae*100):.1f}% lepší než ARIMA")
print(f"a o {((hw_mae-rf_mae)/hw_mae*100):.1f}% lepší než Holt-Winters")

In [ ]:
# Feature importance pro Random Forest
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 10 nejvýznamnějších příznaků pro Random Forest:")
print(feature_importance.head(10))

# Graf feature importance
plt.figure(figsize=(12, 6))
top_features = feature_importance.head(10)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Důležitost příznaku')
plt.title('Top 10 nejdůležitějších příznaků pro Random Forest')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 6. Shrnutí a doporučení

### 📊 Výsledky na testovacím setu (posledních 24 hodin):

| Model | MAE (MWh) | RMSE (MWh) | Interpretace |
|-------|-----------|------------|-------------|
| **Random Forest** | ~143 | ~160 | 🏆 **Nejlepší** - využívá více příznaků |
| **ARIMA** | ~557 | ~618 | Dobrý pro jednoduché vzory |
| **Holt-Winters** | ~747 | ~867 | Problém s komplexní sezónností |

### 🔍 Proč Random Forest vyhrává?

1. **Bohatší feature set**: Používá lag values, rolling statistics, teplotu, čas
2. **Nelineární vztahy**: Dokáže zachytit složité interakce mezi proměnnými
3. **Feature engineering**: Cyklické features (sin/cos) pomáhají s periodicitou
4. **Robustnost**: Ensemble metoda je méně citlivá na outliers

### 🎯 Praktická doporučení:

**Pro produkční použití:**
- **Krátkodobé predikce (1-24h)**: Random Forest s bohatými features
- **Dlouhodobé predikce (týdny)**: ARIMA nebo kombinace modelů
- **Rychlé prototypy**: Holt-Winters (jednoduchá implementace)

**Pro interpretabilitu:**
- **Business prezentace**: ARIMA (jasné trendy/sezónnost)
- **Feature analysis**: Random Forest (feature importance)
- **Automatizace**: Ensemble všech tří modelů

### ⚠️ Důležité poznámky:
- Výsledky závisí na kvalitě a množství dat
- Random Forest potřebuje více historical dat pro feature engineering
- V praxi kombinujte více modelů (ensemble) pro robustnější predikce
- Pravidelně přetrénujte modely na nových datech